https://zhuanlan.zhihu.com/p/139783331

In [1]:
import os
import shutil
import pathlib
from pathlib import Path
from collections import Counter

# 路径获取

## 获取当前工作目录 Path.cwd()

In [2]:
# 虽然在这里打印出来的很像一个字符串，但实际上得到的是一个 WindowsPath('C:\Users\me\study')对象。
# 显示内容由 Path 类的 __repr__ 定义。
Path.cwd()

WindowsPath('d:/ml/code/test/path')

In [3]:
# 如果你只想得到字符串表示，不想要 WindowsPath 对象，可以用 str() 转化
str(Path.cwd())

'd:\\ml\\code\\test\\path'

## 获取用户 home 目录 Path.home()

In [4]:
Path.home()

WindowsPath('C:/Users/Administrator')

## 获取当前文件路径 Path(\_\_file__)

In [5]:
Path(__file__)
# ipynb没有__file__
# 如果可以显示,为 `d:\ml\code\test\path\p.py`

NameError: name '__file__' is not defined

In [6]:
# os获取当前文件名字
os.path.basename(__file__)
# 如果可以显示,为 `p.py`

NameError: name '__file__' is not defined

## 创建任意目录 Path(str)

这里需要注意 2 点：

1. 不管字符串使用的是正斜杠 / 还是反斜杠 \， 在 windows 系统里，得到的路径都是反斜杠\, pathlib 会根据操作系统智能处理。
2. 第二个例子中字符串会被 / 分割，c:d:y 会被当做一个目录名字，pathlib 不会去判断这个文件真的存在哦

In [7]:
Path('subdir/demo_01.py')

WindowsPath('subdir/demo_01.py')

In [8]:
Path('c:d:y/rad.txt')

WindowsPath('c:d:y/rad.txt')

## 获取绝对路径 path.resolve() path.absolute()

In [9]:
Path("archive/demo.txt").resolve()

WindowsPath('D:/ml/code/test/path/archive/demo.txt')

In [10]:
Path("archive/demo.txt").absolute()

WindowsPath('d:/ml/code/test/path/archive/demo.txt')

## 获取文件属性 path.stat()

In [11]:
file = Path("archive/demo.txt")
print(file.stat())
print(file.stat().st_size)
print(file.stat().st_atime)
print(file.stat().st_ctime)
print(file.stat().st_mtime)

os.stat_result(st_mode=33206, st_ino=3377699720531654, st_dev=580483732, st_nlink=1, st_uid=0, st_gid=0, st_size=7, st_atime=1690860323, st_mtime=1690860298, st_ctime=1690860293)
7
1690860323.6089358
1690860293.1397822
1690860298.38083


# 路径组成部分 path.name stem suffix suffixes parent parents parts anchor drive

获取路径的组成部分非常方便：
- .name 文件名，包含后缀名，如果是目录则获取目录名
- .stem 文件名，不包含后缀
- .suffix 后缀，比如 .txt, .png
- .suffixes 多个后缀数组，比如 ['.txt',]
- .parent 父级目录，相当于 cd ..
- .parents 获取所有的上级目录
- .parts 将目录拆开
- .anchor 锚，目录前面的部分 c:\ 或者 /
- .drive 盘符，比如 c:

In [12]:
file = Path("archive/demo.txt").absolute()
file

WindowsPath('d:/ml/code/test/path/archive/demo.txt')

In [13]:
print(file.name)        # 文件名，包含后缀名
print(file.stem)        # 文件名，不包含后缀
print(file.suffix)      # 后缀
print(file.suffixes)    # 后缀

demo.txt
demo
.txt
['.txt']


In [20]:
print(file.parent)      # 父级目录
print()
for i in file.parents:  # 获取所有的上级目录
    print(i)
print()
print(file.parts)       # 目录拆分

d:\ml\code\test\path\archive

d:\ml\code\test\path\archive
d:\ml\code\test\path
d:\ml\code\test
d:\ml\code
d:\ml
d:\

('d:\\', 'ml', 'code', 'test', 'path', 'archive', 'demo.txt')


In [15]:
print(file.anchor)      # 锚，目录前面的部分 C:\ 或者 /
print(file.drive)       # 盘符

d:\
d:


## 相对其他某个路径的结果 path.relative_to()

In [99]:
file = Path("archive/demo.txt")
file.relative_to("archive") # 相对于 archive

WindowsPath('demo.txt')

# path.with_xxx 替换文件名称

## with_name(new_name)

In [18]:
file.with_name("demo1.txt")

WindowsPath('d:/ml/code/test/path/archive/demo1.txt')

## with_stem(new_stem)

In [19]:
file.with_stem("demo2")

WindowsPath('d:/ml/code/test/path/archive/demo2.txt')

## with_suffix(new_suffix)

In [17]:
file.with_suffix(".png")

WindowsPath('d:/ml/code/test/path/archive/demo.png')

# 路径拼接 path / str

In [68]:
path = Path.cwd()

In [69]:
print(path / "archive")
print(path / "archive" / "demo.exe")

d:\ml\code\test\path\archive
d:\ml\code\test\path\archive\demo.exe


In [70]:
print(path.joinpath("archive"))
print(path.joinpath("archive", "demo.exe"))

d:\ml\code\test\path\archive
d:\ml\code\test\path\archive\demo.exe


# 路径测试（判断）

In [71]:
cwd = Path.cwd()
cwd

WindowsPath('d:/ml/code/test/path')

## 是否为文件 path.is_file()

In [72]:
[path for path in cwd.iterdir() if path.is_file()]

[WindowsPath('d:/ml/code/test/path/path.py'),
 WindowsPath('d:/ml/code/test/path/pathlib.ipynb')]

## 是否为文件夹 (目录) path.is_dir()

In [73]:
[path for path in cwd.iterdir() if path.is_dir()]

[WindowsPath('d:/ml/code/test/path/archive')]

## 是否存在 path.exists()

In [74]:
Path("archive/demo.txt").exists()

True

In [75]:
Path("archive/demo1.txt").exists()

True

# 文件操作

## 创建文件 path.torch(exist_ok=True)

In [76]:
Path("archive/demo1.txt").touch(exist_ok=True)

## 创建文件夹 path.mkdir(parents=True, exist_ok=True)

In [77]:
# parents = True: 递归创建目录
# exist_ok= True: 目标目录存在时不报错
Path("archive/middle/demo1").mkdir(parents=True, exist_ok=True)

In [100]:
# 无法递归创建
os.mkdir("archive/middle/demo2")

FileExistsError: [WinError 183] 当文件已存在时，无法创建该文件。: 'archive/middle/demo2'

In [101]:
# 可以递归创建
os.makedirs("archive/middle/demo2", exist_ok=True)

## 删除目录 path.rmdir()
删除目录非常危险，并且没有提示，一定要谨慎操作。一次只删除一级目录，且当前目录必须为空。

In [80]:
Path("archive/middle").rmdir()
# 目录不为空不能删除

OSError: [WinError 145] 目录不是空的。: 'archive\\middle'

In [81]:
Path("archive/middle/demo1").rmdir()

## 删除文件 path.unlink()

In [84]:
Path("archive/demo1.txt").unlink()

## 打开文件 open(file)

In [85]:
with open("archive/demo.txt") as f:
    print(f.read())

1
2
3



In [86]:
# open(file) 可以传入 Path
file_path = Path("archive/demo.txt")
with open(file_path) as f:
    print(f.read())

1
2
3



如果经常使用 pathlib，可以在获取到 Path 路径以后直接调用 path.open() 方法。至于到底用哪一个，其实不必太在意，因为 path.open() 也是调用内置函数 open()

In [98]:
file_path = Path("archive/demo.txt")
with file_path.open() as f:
    print(f.read())

1
2
3



不过 pathlib 对读取和写入进行了简单的封装，不再需要重复去打开文件和管理文件的关闭了。

- .read_text() 读取文本
- .read_bytes() 读取 bytes
- .write_text() 写入文本
- .write_bytes() 写入 tytes

file.write 操作使用的是 w 模式，如果之前已经有文件内容，将会被覆盖

## 移动文件 path.replace(new_path)

移动操作支持的功能很受限。比如当前工作目录如果已经有一个 new_demo.txt 的文件，则里面的内容都会被覆盖。还有，如果需要移动到其他目录下，则该目录必须要存在，否则会报错

In [96]:
file_path = Path("archive/demo.txt")
file_path.replace("new_demo.txt") # 目标文件

WindowsPath('new_demo.txt')

In [97]:
file_path = Path("new_demo.txt")
file_path.replace("archive/demo.txt") # 目标文件

WindowsPath('archive/demo.txt')

为了避免出现同名文件里的内容被覆盖，通常需要进行额外处理。比如判断同名文件不能存在，但是父级目录必须存在；或者判断父级目录不存在时，创建该目录。

```python
ori = Path("archive/demo.txt")
dst = Path("new_demo.txt")
if (not dst.exists()) and dst.parent.exists():
    ori.replace(dest)
```

## 重命名文件 path.with_name(new_name)

In [103]:
file_path = Path("archive/demo.txt")
new_file = file_path.with_name("demo1.txt")
file_path.replace(new_file)

WindowsPath('archive/demo1.txt')

In [104]:
file_path = Path("archive/demo1.txt")
new_file = file_path.with_name("demo.txt")
file_path.replace(new_file)

WindowsPath('archive/demo.txt')

## 重命名文件(不包含后缀名) path.with_stem(new_stem)

In [108]:
file_path = Path("archive/demo.txt")
new_file = file_path.with_stem("demo2")
file_path.replace(new_file)

WindowsPath('archive/demo2.txt')

In [109]:
file_path = Path("archive/demo2.txt")
new_file = file_path.with_stem("demo")
file_path.replace(new_file)

WindowsPath('archive/demo.txt')

## 修改后缀名 path.with_suffix(new_suffix)

In [106]:
file_path = Path("archive/demo.txt")
new_file = file_path.with_suffix(".json")
file_path.replace(new_file)

WindowsPath('archive/demo.json')

In [107]:
file_path = Path("archive/demo.json")
new_file = file_path.with_suffix(".txt")
file_path.replace(new_file)

WindowsPath('archive/demo.txt')

# 子路径扫描
## path.iterdir() 可以扫描某个目录下的所有路径（文件和子目录)， 打印的会是处理过的绝对路径。

In [ ]:
cwd = Path.cwd()
cwd

WindowsPath('d:/ml/code/test/path')

In [ ]:
cwd.iterdir()

<generator object Path.iterdir at 0x00000229B50D11C0>

In [ ]:
for p in cwd.iterdir():
    print(p)

d:\ml\code\test\path\archive
d:\ml\code\test\path\path.py
d:\ml\code\test\path\pathlib.ipynb


## 使用 path.iterdir() 可以统计目录下的不同文件类型

In [ ]:
files = [f.suffix for f in cwd.iterdir() if f.is_file()]
Counter(files)

Counter({'.py': 1, '.ipynb': 1})